# 🚀 Trading Agent - ML Model Training Pipeline

This notebook trains XGBoost models for BTCUSDT across multiple timeframes (15m, 1h, 4h) and packages them for download.

## Sections:
1. Environment Setup
2. Project Setup
3. Configuration
4. Data Fetching & Preparation
5. Model Training
6. Model Download

---
## 1. Environment Setup
Install system dependencies and Python packages

In [ ]:
# Check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("✓ Running in Google Colab")
except:
    IN_COLAB = False
    print("✗ Not running in Google Colab")

In [ ]:
%%bash
# Install TA-Lib system dependencies
echo "Installing TA-Lib..."
wget -q http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
tar -xzf ta-lib-0.4.0-src.tar.gz
cd ta-lib/
./configure --prefix=/usr
make
sudo make install
cd ..
rm -rf ta-lib ta-lib-0.4.0-src.tar.gz
echo "✓ TA-Lib installed successfully"

In [ ]:
# Install Python packages
!pip install -q pandas numpy scikit-learn xgboost joblib
!pip install -q TA-Lib
!pip install -q aiohttp requests python-dateutil pytz pyyaml structlog
!pip install -q matplotlib seaborn plotly tqdm

print("✓ All packages installed successfully")

In [ ]:
# Import libraries
import os
import sys
import time
import zipfile
import getpass
from datetime import datetime, timedelta
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

# ML libraries
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
import joblib
import talib

print("✓ Libraries imported successfully")

---
## 2. Project Setup
Upload project files and configure credentials

In [ ]:
if IN_COLAB:
    from google.colab import files
    import shutil
    
    print("📁 Upload your project as a ZIP file (should contain src/, config/ folders)")
    print("Or you can use git clone if your project is on GitHub\n")
    
    choice = input("Enter '1' to upload ZIP or '2' to clone from git: ").strip()
    
    if choice == '1':
        uploaded = files.upload()
        
        # Extract the uploaded zip
        for filename in uploaded.keys():
            if filename.endswith('.zip'):
                print(f"\nExtracting {filename}...")
                with zipfile.ZipFile(filename, 'r') as zip_ref:
                    zip_ref.extractall('.')
                print("✓ Project extracted")
    else:
        repo_url = input("Enter GitHub repository URL: ").strip()
        
        # Clone to temp directory
        temp_dir = 'temp_project'
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)
        
        !git clone {repo_url} {temp_dir}
        
        # Move files properly using shutil (handles overwriting)
        if os.path.exists(temp_dir):
            print("\n📦 Moving project files...")
            for item in os.listdir(temp_dir):
                if item.startswith('.'):  # Skip hidden files like .git
                    continue
                
                src_path = os.path.join(temp_dir, item)
                dst_path = item
                
                # Remove destination if exists
                if os.path.exists(dst_path):
                    if os.path.isdir(dst_path):
                        shutil.rmtree(dst_path)
                    else:
                        os.remove(dst_path)
                
                # Move from temp to current directory
                shutil.move(src_path, dst_path)
                print(f"  ✅ {item}")
            
            # Clean up temp directory
            shutil.rmtree(temp_dir)
            print("\n✓ Project cloned and files moved successfully")
else:
    print("ℹ Not in Colab - assuming project files are already present")

In [ ]:
# Verify project structure
print("🔍 Verifying project structure...\n")

# Check current directory contents
print("📂 Current directory:", os.getcwd())
print("📋 Directory contents:")
for item in sorted(os.listdir('.')):
    if not item.startswith('.'):
        icon = "📁" if os.path.isdir(item) else "📄"
        print(f"  {icon} {item}")

# Check required directories
print("\n✅ Checking required directories:")
required_dirs = ['src', 'config']
missing_dirs = []

for req_dir in required_dirs:
    if os.path.exists(req_dir):
        print(f"  ✅ {req_dir}/")
        # Show subdirectories
        if os.path.isdir(req_dir):
            subdirs = [d for d in os.listdir(req_dir) if os.path.isdir(os.path.join(req_dir, d)) and not d.startswith('.')]
            for subdir in subdirs:
                print(f"     └── {subdir}/")
    else:
        print(f"  ❌ {req_dir}/ (missing)")
        missing_dirs.append(req_dir)

if missing_dirs:
    print(f"\n❌ ERROR: Missing directories: {missing_dirs}")
    print("\n🔄 SOLUTION:")
    print("1. Go back to the 'Project Upload' cell above")
    print("2. Re-run it and upload/clone your project again")
    print("3. Make sure your project has the correct structure")
    raise FileNotFoundError(f"Missing required directories: {missing_dirs}")
else:
    print("\n✅ Project structure verified!")
    
# Add current directory to Python path
if os.getcwd() not in sys.path:
    sys.path.insert(0, os.getcwd())
    print(f"✅ Added '{os.getcwd()}' to Python path")

# Verify key modules exist
print("\n🔍 Checking key Python modules:")
key_modules = [
    'src/data/delta_client.py',
    'src/data/feature_engineer.py',
    'src/data/data_validator.py',
    'src/ml/xgboost_model.py',
]

all_modules_found = True
for module_path in key_modules:
    exists = os.path.exists(module_path)
    status = "✅" if exists else "❌"
    print(f"  {status} {module_path}")
    if not exists:
        all_modules_found = False

if not all_modules_found:
    print("\n⚠️  Some modules are missing. Check your repository structure.")
    print("   Your project should have been cloned from GitHub.")
else:
    print("\n🎉 All modules found! Ready to proceed.")

print("\n" + "="*60)

In [ ]:
# Create necessary directories
os.makedirs('models', exist_ok=True)
os.makedirs('logs', exist_ok=True)
os.makedirs('plots', exist_ok=True)
print("✓ Created directories: models/, logs/, plots/")

In [ ]:
# Configure Delta Exchange API credentials
print("🔐 Delta Exchange API Configuration\n")

# Try to get from Colab secrets first
api_key = None
api_secret = None

if IN_COLAB:
    try:
        from google.colab import userdata
        api_key = userdata.get('DELTA_API_KEY')
        api_secret = userdata.get('DELTA_API_SECRET')
        print("✓ Loaded credentials from Colab secrets")
    except:
        print("ℹ Colab secrets not found, using manual input")

# Manual input if not found
if not api_key:
    api_key = getpass.getpass('Enter Delta API Key: ')
if not api_secret:
    api_secret = getpass.getpass('Enter Delta API Secret: ')

# Set environment variables
os.environ['DELTA_API_KEY'] = api_key
os.environ['DELTA_API_SECRET'] = api_secret
os.environ['DELTA_API_URL'] = 'https://api.india.delta.exchange'

print("\n✓ API credentials configured")
print("📍 Using Delta Exchange India API")

---
## 3. Configuration
Load configuration and define training parameters

In [ ]:
# Import project modules
from src.data.delta_client import DeltaExchangeClient
from src.data.feature_engineer import FeatureEngineer
from src.data.data_validator import DataValidator
from src.ml.xgboost_model import XGBoostTradingModel
from src.ml.trainer import ModelTrainer

print("✓ Project modules imported successfully")

In [ ]:
# Training configuration
SYMBOL = 'BTCUSD'
TIMEFRAMES = ['15m', '1h', '4h']
HISTORICAL_DAYS = 365  # 1 year of data

# Model parameters
MODEL_PARAMS = {
    'n_estimators': 200,
    'max_depth': 6,
    'learning_rate': 0.1,
    'objective': 'multi:softprob',
    'num_class': 3,
    'eval_metric': 'mlogloss',
    'use_label_encoder': False,
    'random_state': 42,
    'n_jobs': -1
}

# Training parameters
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
TEST_RATIO = 0.15

print("📊 Training Configuration:")
print(f"  Symbol: {SYMBOL}")
print(f"  Timeframes: {TIMEFRAMES}")
print(f"  Historical Data: {HISTORICAL_DAYS} days")
print(f"  Train/Val/Test Split: {TRAIN_RATIO}/{VAL_RATIO}/{TEST_RATIO}")
print(f"\n  Model: XGBoost Classifier")
print(f"  Estimators: {MODEL_PARAMS['n_estimators']}")
print(f"  Max Depth: {MODEL_PARAMS['max_depth']}")
print(f"  Learning Rate: {MODEL_PARAMS['learning_rate']}")

---
## 4. Data Fetching & Preparation
Fetch historical data, validate, and create features

In [ ]:
# Initialize clients
delta_client = DeltaExchangeClient()
feature_engineer = FeatureEngineer()
validator = DataValidator()

print("✓ Initialized data pipeline components")

In [ ]:
def fetch_and_prepare_data(symbol: str, timeframe: str, days: int = 365) -> pd.DataFrame:
    """Fetch and prepare training data for a specific timeframe."""
    print(f"\n📥 Fetching data for {symbol} - {timeframe}...")
    
    # Fetch historical data
    end_date = datetime.utcnow()
    start_date = end_date - timedelta(days=days)
    
    df = delta_client.get_ohlc_candles(
        symbol=symbol,
        resolution=timeframe,
        start=start_date,
        end=end_date,
        limit=days * 100
    )
    
    if df.empty:
        raise ValueError(f"No data fetched for {symbol} - {timeframe}")
    
    print(f"  ✓ Fetched {len(df)} candles")
    
    # Validate and clean
    print(f"  🔍 Validating data quality...")
    df, quality_metrics = validator.validate_and_clean(df)
    print(f"  ✓ Quality score: {quality_metrics.get('quality_score', 0):.2f}/100")
    
    # Create features
    print(f"  🔧 Engineering features...")
    df = feature_engineer.create_features(df)
    print(f"  ✓ Created {len(df.columns)} features")
    
    # Create target variable
    print(f"  🎯 Creating target labels...")
    df = create_target(df, forward_periods=3)
    
    target_dist = df['target'].value_counts().to_dict()
    print(f"  ✓ Target distribution: SELL={target_dist.get(0, 0)}, HOLD={target_dist.get(1, 0)}, BUY={target_dist.get(2, 0)}")
    
    return df


def create_target(df: pd.DataFrame, forward_periods: int = 3) -> pd.DataFrame:
    """Create target variable based on future returns."""
    # Calculate forward returns
    df['future_return'] = df['close'].shift(-forward_periods) / df['close'] - 1
    
    # Define thresholds
    buy_threshold = 0.01   # 1% gain
    sell_threshold = -0.01  # 1% loss
    
    # Create target: 0=SELL, 1=HOLD, 2=BUY
    df['target'] = 1  # Default to HOLD
    df.loc[df['future_return'] > buy_threshold, 'target'] = 2  # BUY
    df.loc[df['future_return'] < sell_threshold, 'target'] = 0  # SELL
    
    # Drop last few rows (no future data)
    df = df.iloc[:-forward_periods]
    
    return df


print("✓ Data preparation functions defined")

In [ ]:
# Fetch data for all timeframes
datasets = {}

for timeframe in tqdm(TIMEFRAMES, desc="Fetching data"):
    try:
        df = fetch_and_prepare_data(SYMBOL, timeframe, HISTORICAL_DAYS)
        datasets[timeframe] = df
    except Exception as e:
        print(f"❌ Error fetching data for {timeframe}: {str(e)}")
        continue

print(f"\n✓ Successfully prepared data for {len(datasets)} timeframes")
for tf, df in datasets.items():
    print(f"  {tf}: {len(df)} samples, {len(df.columns)} features")

---
## 5. Model Training
Train XGBoost models for each timeframe

In [ ]:
def split_data(df: pd.DataFrame, train_ratio: float = 0.7, val_ratio: float = 0.15) -> Tuple:
    """Split data into train, validation, and test sets."""
    total = len(df)
    train_end = int(total * train_ratio)
    val_end = int(total * (train_ratio + val_ratio))
    
    train_df = df.iloc[:train_end]
    val_df = df.iloc[train_end:val_end]
    test_df = df.iloc[val_end:]
    
    return train_df, val_df, test_df


def prepare_for_model(df: pd.DataFrame, target_col: str = 'target') -> Tuple:
    """Prepare features and target for model training."""
    # Exclude columns
    exclude_cols = ['timestamp', 'symbol', 'timeframe', 'open', 'high', 'low', 'close', 'volume', 
                   'future_return', target_col]
    feature_cols = [col for col in df.columns if col not in exclude_cols]
    
    X = df[feature_cols].copy()
    y = df[target_col].copy()
    
    # Replace inf with nan and fill
    X = X.replace([np.inf, -np.inf], np.nan)
    X = X.fillna(method='ffill').fillna(0)
    
    return X, y, feature_cols


print("✓ Training helper functions defined")

In [ ]:
# Training loop
trained_models = {}
training_results = []

for timeframe in tqdm(TIMEFRAMES, desc="Training models"):
    if timeframe not in datasets:
        print(f"⚠ Skipping {timeframe} - no data available")
        continue
    
    print(f"\n{'='*60}")
    print(f"🤖 Training model for {SYMBOL} - {timeframe}")
    print(f"{'='*60}")
    
    df = datasets[timeframe]
    start_time = time.time()
    
    # Split data
    train_df, val_df, test_df = split_data(df, TRAIN_RATIO, VAL_RATIO)
    print(f"\n📊 Data split: Train={len(train_df)}, Val={len(val_df)}, Test={len(test_df)}")
    
    # Prepare features and target
    X_train, y_train, feature_names = prepare_for_model(train_df, 'target')
    X_val, y_val, _ = prepare_for_model(val_df, 'target')
    X_test, y_test, _ = prepare_for_model(test_df, 'target')
    
    print(f"✓ Features prepared: {len(feature_names)} features")
    
    # Create and train model
    print(f"\n🔄 Training XGBoost model...")
    model = XGBoostTradingModel()
    
    # Store feature names
    model.feature_names = feature_names
    
    # Scale features
    model.scaler = StandardScaler()
    X_train_scaled = model.scaler.fit_transform(X_train)
    X_val_scaled = model.scaler.transform(X_val)
    X_test_scaled = model.scaler.transform(X_test)
    
    # Create XGBoost model
    model.model = XGBClassifier(**MODEL_PARAMS)
    
    # Train with validation
    model.model.fit(
        X_train_scaled,
        y_train,
        eval_set=[(X_train_scaled, y_train), (X_val_scaled, y_val)],
        verbose=False
    )
    
    # Evaluate
    train_pred = model.model.predict(X_train_scaled)
    val_pred = model.model.predict(X_val_scaled)
    test_pred = model.model.predict(X_test_scaled)
    
    train_acc = (train_pred == y_train).mean()
    val_acc = (val_pred == y_val).mean()
    test_acc = (test_pred == y_test).mean()
    
    training_time = time.time() - start_time
    
    print(f"\n📈 Results:")
    print(f"  Train Accuracy: {train_acc:.4f}")
    print(f"  Val Accuracy:   {val_acc:.4f}")
    print(f"  Test Accuracy:  {test_acc:.4f}")
    print(f"  Training Time:  {training_time:.2f}s")
    
    # Get feature importance
    feature_importance = pd.DataFrame({
        'feature': feature_names,
        'importance': model.model.feature_importances_
    }).sort_values('importance', ascending=False).head(10)
    
    print(f"\n🎯 Top 10 Features:")
    for idx, row in feature_importance.iterrows():
        print(f"  {row['feature']:<30} {row['importance']:.4f}")
    
    # Save model
    model_filename = f"models/xgboost_{SYMBOL}_{timeframe}.pkl"
    model.save(model_filename)
    print(f"\n💾 Model saved: {model_filename}")
    
    # Store results
    trained_models[timeframe] = model
    training_results.append({
        'timeframe': timeframe,
        'train_accuracy': train_acc,
        'val_accuracy': val_acc,
        'test_accuracy': test_acc,
        'training_time': training_time,
        'num_features': len(feature_names),
        'train_samples': len(train_df),
        'model_file': model_filename
    })

print(f"\n\n{'='*60}")
print(f"✓ Training complete for {len(trained_models)} models")
print(f"{'='*60}")

In [ ]:
# Display summary table
results_df = pd.DataFrame(training_results)

print("\n📊 Training Summary:")
print("=" * 100)
print(results_df.to_string(index=False))
print("=" * 100)

# Calculate averages
print(f"\n📈 Average Metrics:")
print(f"  Train Accuracy: {results_df['train_accuracy'].mean():.4f}")
print(f"  Val Accuracy:   {results_df['val_accuracy'].mean():.4f}")
print(f"  Test Accuracy:  {results_df['test_accuracy'].mean():.4f}")
print(f"  Training Time:  {results_df['training_time'].mean():.2f}s")

In [ ]:
# Visualize results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy comparison
results_df.set_index('timeframe')[['train_accuracy', 'val_accuracy', 'test_accuracy']].plot(
    kind='bar', ax=axes[0], rot=0
)
axes[0].set_title('Model Accuracy by Timeframe', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Accuracy')
axes[0].set_xlabel('Timeframe')
axes[0].legend(['Train', 'Validation', 'Test'])
axes[0].grid(axis='y', alpha=0.3)

# Training time
results_df.set_index('timeframe')['training_time'].plot(
    kind='bar', ax=axes[1], color='coral', rot=0
)
axes[1].set_title('Training Time by Timeframe', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Time (seconds)')
axes[1].set_xlabel('Timeframe')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('plots/training_summary.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Visualization saved to plots/training_summary.png")

---
## 6. Model Download
Package all trained models and download as ZIP file

In [ ]:
# List all model files
model_files = list(Path('models').glob('*.pkl'))

print(f"📦 Found {len(model_files)} model files:")
for model_file in model_files:
    size_mb = model_file.stat().st_size / (1024 * 1024)
    print(f"  • {model_file.name} ({size_mb:.2f} MB)")

In [ ]:
# Create ZIP file with all models
zip_filename = f'trained_models_{SYMBOL}_{datetime.now().strftime("%Y%m%d_%H%M%S")}.zip'

print(f"\n📦 Creating ZIP archive: {zip_filename}")

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add model files
    for model_file in model_files:
        zipf.write(model_file, arcname=f"models/{model_file.name}")
        print(f"  ✓ Added {model_file.name}")
    
    # Add training summary
    results_df.to_csv('training_summary.csv', index=False)
    zipf.write('training_summary.csv', arcname='training_summary.csv')
    print(f"  ✓ Added training_summary.csv")
    
    # Add plot if exists
    if os.path.exists('plots/training_summary.png'):
        zipf.write('plots/training_summary.png', arcname='training_summary.png')
        print(f"  ✓ Added training_summary.png")

zip_size_mb = Path(zip_filename).stat().st_size / (1024 * 1024)
print(f"\n✓ ZIP archive created: {zip_filename} ({zip_size_mb:.2f} MB)")

In [ ]:
# Download the ZIP file
if IN_COLAB:
    from google.colab import files
    print(f"\n⬇️ Downloading {zip_filename} to your local machine...")
    files.download(zip_filename)
    print("✓ Download started! Check your browser downloads.")
else:
    print(f"\n✓ ZIP file ready: {zip_filename}")
    print(f"  You can find it in the current directory")

---
## 🎉 Training Complete!

### What's Next?

1. **Extract the downloaded ZIP file** on your local machine
2. **Copy the model files** to your project's `models/` directory
3. **Update your config** to use the new model paths
4. **Run predictions** using the trained models

### Model Files:
- `xgboost_BTCUSDT_15m.pkl` - 15-minute timeframe model
- `xgboost_BTCUSDT_1h.pkl` - 1-hour timeframe model  
- `xgboost_BTCUSDT_4h.pkl` - 4-hour timeframe model

### Additional Files:
- `training_summary.csv` - Training metrics and details
- `training_summary.png` - Performance visualization

---

**Happy Trading! 📈**